# 1次拡散方程式を陰解法で解く

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
import glob

In [2]:
dt = 0.01
dx = 0.001
tmax = 10 # [s]
tnum = int(tmax / dt)
print("tmax: ", tmax, "tnum: ", tnum)

xmin, xmax = 0, 0.1
xnum = int((xmax - xmin) / dx + 1)
xs = np.linspace(xmin, xmax, xnum)
us = np.zeros_like(xs)

init_temp = 60
us[:] = init_temp

tmax:  10 tnum:  1000


In [3]:
def plot(us, it):
    plt.plot(us[:], xs)
    plt.ylabel("x [m]")
    plt.xlabel("temperature")
    plt.xlim(-1, 100)
    plt.ylim(0, 0.1)

def save_fig(us, it, path, ice_h):
    fig = plt.figure(facecolor="white", figsize=(5, 8))
    plt.plot(us[:], xs, label="temperature")
    plt.xlabel("temperature")
    plt.axvspan(-1, 100, 0, ice_h * 10, color = "gray", label="ice")
    plt.ylabel("x [m]")
    plt.ylim(0, 0.1)
    plt.title("t={0} [s]".format(np.round(it * dt, 10)))
    plt.xlim(-1, 100)
    plt.legend()
    fig.savefig(path)
    plt.close()

In [4]:
mu = 0.117  # 熱拡散係数 [m^2/s]  銅 0.117, 鉄0.022

# パイプが沈み込む速度(等速)
v_pipe = 0.001  # [m/s]

r = mu * dt / (dx**2)

In [5]:
us[0] = 0 

In [6]:
!rm img/*png

In [7]:
for it in tqdm(range(0, tnum)):
	a = np.zeros((len(xs), len(xs)))
	# set matrix
	x_bound = v_pipe * dt * it  # 氷の位置
	is_ice = xs < x_bound
	is_ice_index = np.argmin(is_ice)
	a[0, 0], a[-1, -1] = 1.0, 1.0
	us[is_ice] = 0.0
	a[is_ice, is_ice] = 1.0
	for i in range(is_ice_index, len(xs) -1):
		a[i, i-1] = -r
		a[i, i] = 1 + 2 * r
		a[i, i + 1] = -r

	inv_a = np.linalg.pinv(a)
	new_us = np.zeros_like(us)
	if it != 0:
		new_us = np.dot(inv_a, us)
		us = new_us
		save_fig(us=new_us, it=it, path="img/it={0:06}".format(it), ice_h=x_bound)

100%|██████████| 1000/1000 [01:26<00:00, 11.56it/s]


In [8]:
files = sorted(glob.glob('img/*.png'))[::10]
images = list(map(lambda file : Image.open(file) , files))
images[0].save('copper.gif', save_all=True, \
    append_images=images[1:], fps=100, loop=0)

In [9]:
from IPython.display import HTML
HTML('<img src="SUS.gif">')

## 結果
灰色の網掛けは氷。一定速度で氷に入っていった場合を可視化した。横軸は温度、縦軸は長さ。温度勾配が生じる。